# Let's import the necessary functions that we will use throughout the exercises

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
%matplotlib inline
%matplotlib notebook

# The Cellular Automata
We will now start looking into what is a <b>Cellular Automata (CA)</b> and what it is possible to be done with it.

During the presentation we have seen that in order to define a CA it is necessary to define a few concepts:
- the <b>space</b> that we are going to divide into unitary cells
- the number and types of possible <b>states</b> (for example 0 and 1, to indicate a cell that is "full" or "empty") of each unitary cells
- the <b>rule</b> that updates the cells at every iteration
- the <b>neighbourhood</b> (i.e. the number of neighboring cells) that is taken into consideration to enforce the rule

## Let's start with the space
We are going to consider a 2D space that is divided into unitary cells. This can easily be visualized as a squared paper, like the one shown below, where each square can have values that are either 1 or 0.

<img src="images/squaredPaper.jpg" width="250" height="250" />

This will be used throughout all of the examples, so the space will always be a 2D space divided into unitary cells.

We will use <b>periodic boundary conditions</b> for the first part. This means that when we are at one of the extremes of the squared grid, the unitary cell will interact with the cell that is at the other extreme, as shown in the figure.

<img src="images/periodicBoundary.png" width="262" height="214" />

Which means, as you have seen in the presentation, that we are working on a torus. Can you understand why?

<img src="images/grid-to-torus.jpeg" width="408" height="144" />


## And the values that each unitary cell can have
The values that each cell can take will be 0 or 1 for the simple examples, but can differ once we start introducing more complex models.

In the notebook we are going to use a <b>matrix</b> in order to make this "square grid". A matrix can be visualized as a 2D table, where each square has a value. Normally one can use any value, here we are going to restrict the values of the matrix to those that are allowed by the CA that we are using. 

In [ ]:
def makeMatrix(nx=100, ny=100, initialization="random", concentration=0.5):
    """This function is used to create a matrix with the desired configuration. The easiest example is to create 
    a matrix of single values through the numpy functions np.zeros or np.ones. Other initialization can be created
    at will, given a good implementation."""
    
    if initialization == "zeros":
        matrix = np.zeros((ny,nx))
        
    elif initialization == "ones":
        pass # Your code goes here, instead of pass
    
    elif initialization == "random":
        pass # Your code goes here, instead of pass
    
    else:
        print("Inconnu")
    return matrix

## Define the neighborhood
Now we define the neighborhood used to update the cells. This means that we are going to select which neighboring cells are taken into account every iteration to decide the next value of a cell.

Within this work we are going to consider only two possible neighborhoods: <b>Von Neumann</b> and <b>Moore</b>.

### Von Neumann neighborhood
This is the simplest case we can consider. Here we only consider the East, North, West, South neighboring cells. It means that only these cells will be taken into consideration when deciding the next value that the central cell will have after 1 timestep

### Moore neighborhood
In this case we add 4 more cells: North-East, North-West, South-East, South-West. Therefore all those cells that surround the current one. 

<img src="images/neighborhoods.gif" width="389" height="260" />

The way we define the function *giveNeighborhood* might appear strange, but it helps later in the code.

We imagine to sit on a cell in the *space* that we have defined and we want to return all of the distances of the neighbors relative to the chosen cell. This means that to return the right and left neighbors, we have to return _[[0,1],[0,-1]]_ in the for of a list of lists.

In [ ]:
def giveNeighborhood(neighType="Von_Neumann"):
    """This function returns a list of neighbors as list of the deltas from the given cell."""
    
    if neighType == "Von_Neumann":
        return [[-1,0],[0,-1],[1,0],[0,1]]
    elif neighType == "Moore": # This can be left as an exercise to the student, after seeing the previous.
        return [[-1,-1],[-1,0],[-1,1],[0,-1],[0,1],[1,-1],[1,0],[1,1]]
    else:
        raise NameError("The neighborhood can be only Von_Neumann or Moore. Otherwise implement a new one")

## Continue with the rules
Now we introduce possible rules that determine how the cells of the CA evolve in time.

We will start with a simple rule, to have a look at a CA in action.

The *Parity Rule* is already implemented in the next cell. The value of each cell is decided depending on the number of "ones" in the surrounding cells. If the number is even, then the cell will take the value of 0. If the number is odd, then it will take the value of 1.
Since the values can only be "1" or "0", the simple implementation is to sum the values of all the neighbors and check if it is even or odd.

In this case it is evident that the choice of the neighborhood can have an important role for the rule. But we will add this later.

In [ ]:
def ruleIterate(matrix,rulename,neighborhood="Von_Neumann"):
    """This is the most important function. Here the rules will be specified and an algorithm that enforces them 
    must be written. It is very important to have a clear understanding of what elements are given and which are 
    expected to be returned.
    
    """
    newMatrix = matrix.copy()
    
    leny = matrix.shape[0]
    lenx = matrix.shape[1]
    
    neighborList = giveNeighborhood(neighType=neighborhood)
    
    if rulename == "ParityRule":
        for ix in range(lenx):
            for iy in range(leny):
                
                neighborsSum = 0
                for delta in neighborList:
                    [idx,idy] = (np.array([ix,iy]) + np.array(delta)) % np.array([lenx,leny])
                    neighborsSum += matrix[idx,idy]
                    
                if neighborsSum % 2 == 0: # If the sum of the neighbors is even...
                    newMatrix[ix,iy] = 0  # The cell becomes 0
                else:                     # If the sum is odd...
                    newMatrix[ix,iy] = 1  # The cell becomes 1

    elif rulename == "ConwayRule":
        pass # Your code goes here, instead of pass
                        
    elif rulename == "MyRule":
        pass # Your code goes here, instead of pass
                            
    else: raise NameError("Règle inconnu")
                        
    return newMatrix

### This is an helper function, needed to animate the CA
It is a bit complicated to implement and it is out of the scope of the exercise, therefore this function is given and should not be changed at any time.

In [ ]:
def RunAndShow(cellularAutomata,rulename="ConwayRule",neighborhood="Moore",niter=10):
    """This function must be called when one wants to run and visualize a CA. It takes an initialized matrix as an 
    input and returns the """
    
    [ny,nx] = cellularAutomata.shape[0],cellularAutomata.shape[1]
    cellmatrix = makeMatrix(nx,ny,initialization="random")

    fig = plt.figure(figsize= (8,8))
    plt.axis([-0.5,nx-0.5,-0.5,ny-0.5])
    im = plt.imshow(cellmatrix,animated=True)
    plt.colorbar()

    def runstep(i):
        """This function is called at each step when creating the animation"""
        global cellmatrix
        if i==0:
            cellmatrix = cellularAutomata.copy()
        else:
            cellmatrix = ruleIterate(cellmatrix,rulename,neighborhood)
        im.set_array(cellmatrix)
        return im

    # Here we create the animation. It can be saved or showed on the notebook using the HTML plugin
    # For now we use the HTML plugin (you can find it later)
    ani = animation.FuncAnimation(fig, runstep, frames=niter, interval=250, repeat=False)
    plt.close()
    return ani

# The first example
## Parity rule with Von Neumann neighborhood

Let's begin with our first CA. The Parity rule has been explained previously, so here we will see how it works in a "real" case. We will test it on a simple square lattice where all the values are zeros but for a small square of ones in the middle of the canvas.

In [ ]:
[nx,ny] = [30,30]
maxiter = 50

# We are initializing a cell of zeros with a central square of ones
cellularAutomata = makeMatrix(nx, ny, initialization="zeros")
cellularAutomata[14:16,14:16] = 1

ani = RunAndShow(cellularAutomata,rulename="ParityRule",neighborhood="Von_Neumann",niter=maxiter)
HTML(ani.to_jshtml()) # Here the gif is shown

While this is a very beautiful pattern, what would happen with other initial configurations?

### Change the initial configurations

In order to see what happens in different initial conditions try to do two things:

 - Start again from a matrix made of only "zeros", then change some of the values of the matrix to "ones". Then run the code and see what happens
 - Go back to the <b>makeMatrix</b> function and add different initialization schemes, such as a "random" one, where 0s and 1s are randomly assigned on the grid.

### Random initial configuration: discussion

Unfortunately, when starting with a random initial configuration no global behaviour can be observed. There is no evolution and (likely) the CA tends towards a 50/50 division of the cells. Therefore we should look into another rule, but which one?

# How many rules are possible?

If you have an interest in the calculation of combinations, you will like this small exercise. 
Can you say how many rules are possible for a given neighborhood?

In order to compute them it might help to do a few steps first. It should be noted that you can do this exercise even just by *brute force*, writing on paper all of the possible combinations, if you use only 2 neighbours as a start.

You can start your reasoning from here:

 - define a neighborhood, i.e. the number of cells you are considering when evaluating for the next state of the central one (take 2 if you want to do it *brute force*)
 - how many different states each of these cell can have?
 - how many different combinations of states exist? (note that (1,0,0,0) is different from (0,1,0,0) so each one must be counted)
 
Have you counted the number of combinations or have you found a formula to calculate the combinations? If you have found a formula, see how different the number of combinations is for the Moore or Von Neumann neighborhoods.

In order to create a rule, however, we have to establish what will be the *next* value of the current cell. 

 - Take all of the combinations of states that you have calculated before. 
 - Imagine that *ALL* of those combinations yield 0 as next state of the cell. This counts as 1 rule.
 - Now imagine that all yield 0, but *ONE* yields a 1. How many different ways this can be achieved?

You should repeat this reasoning for all possible combinations, with all possible outcomes. The numbers you will be calculating will be huge! Try it for the two neighborhoods.

It would obviously impossible to explore all of the possible rules. Also, most of these rules will quickly result in complete barren environment in all of the cells, but some other rules might give birth to some interesting pattern. Here we will look into a few interesting rules.

# Conway's rule of life

This rule was introduced in 1970 as a game on a mathematical magazine. Many readers tried to play the game (mostly with pen and paper, as the computers were not commons at the time) and found many self-sustaining configurations and even some "evolving" ones. This sparked an interest on the topic and several researchers started working on this "game".

The rule is set to represent the "life" of some sort of cellular entity (imagine cells or bacteria). The main idea is that cells that are left alone would die, because alone are not able to sustain themselves. On the other hand, if too many cells are full, then there is not enough food for everybody, and some of the cells end up dying.
Cells survive only if they are surrounded by 2 or 3 neighbors and they can even multiply, if they are not overcrowded or completely alone. The idea, therefore is that:

    if CELL is alive:
        0-1 neighbors --> cell dies because it is alone
        2-3 neighbors --> cell lives
        >4 neighbors --> cell dies because it is overcrowded and there is not enough food for everybody
    if CELL is dead:
        3 neighbors --> cell becomes alive (imagine mitosis of a cell)
        every other case --> cell remains dead
        
### Implement the rule of life

 - Go back to the <b>ruleIterate</b> function and try to implement the rules stated above. Feel free to re-use parts of the parityRule already implemented and modify it to give the expected outcome
 - After you have implemented the rule, try to run the following examples to test whether it works as intended

## Still life
Let's start by using the same initial configuration that created that nice pattern before, with the parity rule. What will happen now?

In [ ]:
[nx,ny] = [30,30]
maxiter = 10

# This is again the square of ones in the middle
cellularAutomata = makeMatrix(nx, ny, initialization="zeros")
cellularAutomata[14:16,14:16] = 1

ani = RunAndShow(cellularAutomata,rulename="ConwayRule",neighborhood="Moore",niter=maxiter)
HTML(ani.to_jshtml())

Unfortunately in this case the final result is a so-called "still life", where nothing happens. If your simulation shows a square that doesn't move, then it is correct!

Luckily there are some more interesting configurations to be studied.

## Glider
One of the first configurations to be found that was self-sustaining is the "glider", a sort of small object that remains alive forever and keeps moving. This is one of the most frequent configurations that is found when using a "random" initial configuration for the matrix.

In [ ]:
[nx,ny] = [10,10]
maxiter = 100

# This is a "spaceship"
cellularAutomata = makeMatrix(nx, ny, initialization="zeros")
cellularAutomata[0,3:6] = cellularAutomata[1,5] = cellularAutomata[2,4] = 1


ani = RunAndShow(cellularAutomata,rulename="ConwayRule",neighborhood="Moore",niter=maxiter)
HTML(ani.to_jshtml())


However there are a number of other configurations that can be created. This rule has been extensively studied and many interesting states have been found.

A list of possible patterns can be found here: 
 - https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life
 - http://pentadecathlon.com/lifenews/2006/06/single_glider_collision_survey.html
 - http://mathworld.wolfram.com/GameofLife.html

And a super comprehensive list can be found here: 
 - http://conwaylife.com/wiki/Category:Patterns
 
Also, feel free to use the *random* initial configuration that you implemented before. This time it will be much more interesting!

# Implementing other rules

Are there any other interesting rules? Sure!

While "Life" is probably the most studied rule, there are a number of other rules that one can implement to have fun.

Here is a quite comprehensive list of interesting rules that you can have fun implementing on your own: http://psoup.math.wisc.edu/mcell/rullex_life.html

 - Go back to the <b>ruleIterate</b> function and implement one of the rules described in the link above in the place where "myRule" is given. Feel free to add even more than just one more rule!

# What have we seen?

Most of the rules that we have seen form extremely interesting patterns, but they can hardly be used to model real phenomena. Therefore we will now turn to different rules that will allow us to study properties actually related to materials.

# A new model: grain growth

The solidification of a material is a complex process, very difficult to model accurately. In the field of metallurgy it is important to understand how solidification happens, since this will impact the mechanical properties of the material. 

The following image shows 4, among the many, possible microscopic configurations of steel, after it has been cooled down in different conditions. Each one of them has different properties and is used for different purposes. Knowing exactly which configuration is growing when casting steel is important to obtain a material with specific properties.
<img src="images/ironPhases.png" width="1000" height="280" />

## Atoms and phases of matter

The smallest constitutive unit of matter that we consider for materials is the atom, which can, for simplicity, be approximated by a sphere. Atoms can arrange in various ways, and different phases can exist. Normally we talk about gas (atoms don't have any structure), liquid (atoms are disordered, but stay close to each other), and solid (ordered and packed).

<img src="images/statesofmatter.jpg" width="600" height="248" />

## Solid phase: crystals and grains

However, even when all of the atoms are in the same phase, they can still show some differences. Atoms in the solid phase can arrange themselves differently, as it can be seen in the following figure, where 3 common solid arrangements of atoms are shown:

<img src="images/scfccbcc.png" width="559" height="238" />

The same material, in different conditions, can show any of these (and many other!) configurations. Changes in chemical composition, pressure, temperature can drive the material from one structure to another. However, for the rest of the lecture, we will assume that all atoms have the same configuration (e.g. FCC) and not consider differences in this sense.

A group of atoms sharing the same arrangement and direction is called a <b>crystal</b>. Crystals are formed upon solidifications, when the atoms begin to re-arrange themselves in an ordered manner. However we talk about a "crystalline" system only when all of the atoms have the same orientation, which is hardly ever the case. It can happen that the atoms will have different orientations, different general directions, even if their internal arrangement is the same. In this case we talk about a polycristalline structure and we introduce the concept of grain.

<img src="images/crystal-poly.png" width="325" height="200" />

A <b>grain</b> is a single crystal surrounded by other crystals of the same type, but at different orientations. Multiple grains, each oriented in a different manner, form the solid as we know it. The orientation, form and dimension of the grains are controlled by many factors, such as the cooling rate (the velocity at which the temperature is lowered) and the amount of other elements in the material, and affects the mechanical properties of the solid.

In the following figure we can clearly see different grains, i.e. different groups of atoms that oriented in the same manner. The border between two crystals is called "grain boundary" and it is also important to know how it is shaped and the angle between different grains.
<img src="images/grains-HRTEM-2.png" width="454" height="287" />

Therefore it is necessary to study how different parameters can influence the microscopic picture, in order to drive the solidification towards the system with the most interesting mechanical properties.

## Solidification

Solidification is the process or re-arrangement of the atoms from a very chaotic state to a very compact and ordinated one.

At microscopic level it is very complex and till today remains one of the biggest challenges in computational materials science. There are two main concept to understand in order to make a reasonable model:

   - Solidification is an activated process
   - Solidification is driven by temperature
    

### Activated process

Solidification does not happen at the same time everywhere, as the atoms do not tend to spontaneously re-arrange in a solid form on their own. It is usually necessary to wait for an "event" which creates a <b>seed</b> for the solidification. What happens in experiments is that a very small part of the liquid becomes solid (either spontaneously or due to some type of interaction) and forms a (small) solid seed. After this, all the liquid attached to it will start to solidify as well.

<img src="images/GrainGrowth.gif" width="226" height="198" />

### Driven by temperature

It is normally said that a liquid, once it has reached the solidification temperature, will become solid. However, this is not completely true. What really happens is that the liquid can be cooled beyond the transition temperature, before starting to solidify.

The lower is the temperature of the liquid, the higher the probability that a seed will appear and will start to solidify together with its surrounding atoms. Therefore a seed that appears early will have more time to solidify the liquid, growing bigger than other grains.

## Modelling grain growth with a CA

Keeping in mind these two concepts, we can imagine what is necessary in order to provide a reasonable model of solidification, using a CA. We have to introduce a seed in the system, from which the solidification will have origin, at a temperature that is different for each seed. This will change the dimension and shape of the grain during its growth.

Grains differ one from the other in terms of relative orientation of the atoms and this will be highlighted by using different colors for each grain. In reality each one, with its own orientation, will grow only following a certain direction and shape, however in this simple model we are going to disregard that.

In this simple model we are going to assume that the temperature at which seeds appear at the border and in the center of the liquid can differ. This reflects the fact that the liquid can interact with external factors that will make the seed appear first in either region.

The grains will grow by "capturing" the neighboring cells. The velocity of growth depends on the local supercooling of the liquid, which keeps increasing over time (as the liquid gets colder and colder).

Differently from before, the CA does not have periodic boundary conditions and cells at the border will not interact with those at the other side.

# These functions are now given, all the rules have already been implemented.

In [ ]:
def ruleIterategg(matrix, shadowMatrix, undercoolingMatrix, seedMatrix, timestep=0.001, ngrains=42): 
    newMatrix = matrix.copy()
    leny = matrix.shape[0]
    lenx = matrix.shape[1]
    
    neighborList = [[-1,0],[0,-1],[1,0],[0,1]]
    
    # Just consider the cells that are non zero, i.e. the solid ones
    grainy, grainx = matrix.nonzero()
    # Iterate over the non zero cells to propagate them
    for _idf in range(len(grainx)):
        idgrain = int(matrix[grainy[_idf],grainx[_idf]]-1)
        for delta in neighborList:
            [idx,idy] = (np.array([grainx[_idf],grainy[_idf]]) + np.array(delta))
            if (0 <= idx < lenx) and (0 <= idy < leny) and matrix[idy,idx] == 0:
                shadowMatrix[idy,idx,idgrain] += (undercoolingMatrix[grainy[_idf],grainx[_idf]]**3 \
                                                  + undercoolingMatrix[grainy[_idf],grainx[_idf]]**2)*timestep
                if shadowMatrix[idy,idx,idgrain] >=1:
                    newMatrix[idy,idx] = idgrain+1
                    seedMatrix[idy,idx] = 0
    
    # The non zero cells in the seed matrix
    seedy, seedx = seedMatrix.nonzero()
    # If now they are able to spawn, then make them spawn
    for _ids in range(len(seedx)):
        if undercoolingMatrix[seedy[_ids],seedx[_ids]] > seedMatrix[seedy[_ids],seedx[_ids]]:
            newMatrix[seedy[_ids],seedx[_ids]] = np.random.randint(1,ngrains+1)
            seedMatrix[seedy[_ids],seedx[_ids]] = 0

    return newMatrix, shadowMatrix

In [ ]:
def RunAndShowgg(nx,ny,\
                 nGrainsSurface,deltaTSurfaceMean,deltaTSurfaceSigma,\
                 nGrainsVolume,deltaTVolumeMean,deltaTVolumeSigma, \
                 undercoolingVelocity, niter=10):
    
    global shadowMatrix
    global undercoolingMatrix
    
    # We initialize the necessary matrixes that we are going to use in the program
    cellularAutomata = makeMatrix(nx,ny,initialization="zeros")
    seedMatrix = makeMatrix(nx,ny,initialization="zeros")
    undercoolingMatrix = makeMatrix(nx,ny,initialization="zeros")
    orientations = 42
    
    # Here I am placing the seeds in random positions of the matrix
    # First we do on the surface
    for _idGrain in range(nGrainsSurface):
        if np.random.randint(2):
            idx = np.random.randint(0,nx-1)
            idy = np.random.choice([0,ny-1])
        else:
            idx = np.random.choice([0,nx-1])
            idy = np.random.randint(0,ny-1)
        if seedMatrix[idy,idx] > 0:
            seedMatrix[idy,idx] = np.min([seedMatrix[idy,idx],\
                                          np.random.normal(loc=deltaTSurfaceMean,scale=deltaTSurfaceSigma)])
        else:            
            seedMatrix[idy,idx] = np.random.normal(loc=deltaTSurfaceMean,scale=deltaTSurfaceSigma)
    
    # Then we do it in the bulk
    for _idGrain in range(nGrainsVolume):
        idx = np.random.randint(int(nx*.15),int(nx*.85))
        idy = np.random.randint(int(ny*.15),int(ny*.85))
        if seedMatrix[idy,idx] > 0:
            seedMatrix[idy,idx] = np.min([seedMatrix[idy,idx],\
                                         np.random.normal(loc=deltaTVolumeMean,scale=deltaTVolumeSigma)])
        else:            
            seedMatrix[idy,idx] = np.random.normal(loc=deltaTVolumeMean,scale=deltaTVolumeSigma)
        
    # Inizialization of the cellmatrix to guarantee that it spans all of the colourbar
    cellmatrix = makeMatrix(nx,ny,initialization="random")*orientations
    # Inizialization of the shadowMatrix that will contain the values of the 
    shadowMatrix = np.zeros((nx,ny,orientations+1))

    fig = plt.figure(figsize= (8,8))
    im = plt.imshow(cellmatrix,animated=True,cmap="Spectral")
    plt.title("Supercooling = {} K".format(np.min(undercoolingMatrix)))
    plt.colorbar()
    
    timestep = np.abs(1/(undercoolingVelocity*niter)**3)

    def runstep(i):
        global cellmatrix
        global shadowMatrix
        global undercoolingMatrix

        if i==0: # Starting frame
            cellmatrix = cellularAutomata.copy()
            
        else: # Everything else
            undercoolingMatrix -= undercoolingVelocity
            cellmatrix, shadowMatrix = ruleIterategg(cellmatrix,shadowMatrix,undercoolingMatrix,\
                                                     seedMatrix,timestep,ngrains=orientations)
        #im.title("Supercooling = {} K".format(np.min(undercoolingMatrix)))
        im.set_data(cellmatrix)
        return im


    ani = animation.FuncAnimation(fig, runstep, frames=niter, interval = 80, repeat=False)
    plt.close()
    return ani

## Now we can play around with the parameters that affect the simulation

A few examples of interesting configurations are here provided, in order to show some patterns that can be created using this model.

The parameters that can be freely modified are:

   - <b>nx,ny</b> --> the dimension of the grid. The bigger the grid, the longer the simulation will take to evaluate
   - <b>maxiter</b> --> the number of iterations of the simulation. It is useless to set it to a number greater than nx/2, because anyway the simulation will come to a stop. The longer it is, the more time it will take to evaluate
   - <b>nGrainsSurface</b> --> the number of grains that must be randomly placed on the border of the grid
   - <b>nGrainsVolume</b> --> the number of grains that must be randomly placed in the center of the grid
   - <b>deltaTSurfaceMean</b> --> the average temperature of the grains on the border
   - <b>deltaTSurfaceSigma</b> --> how far can the temperature can go from the mean for the seeds on the border
   - <b>deltaTVolumeMean</b> --> the average temperature of the grains in the center
   - <b>deltaTVolumeSigma</b> --> how far can the temperature can go from the mean for the seeds in the center
   - <b>undercoolingVelocity</b> --> the cooling ratio of the liquid
    
### Case one: seeds appear first on the surface

In this case we are going to set the parameters so that the seeds will appear first on the surface and only after in the center. This is a common scenario, because the seeds tend to form first where they are in contact with something solid (like the container where it is casted)

How do you think the grains will look like?

In [ ]:
[nx,ny] = [100,100]
maxiter = 120
nGrainsSurface = 100
nGrainsVolume = 100
deltaTSurfaceMean = 0.5
deltaTSurfaceSigma = 0.2
deltaTVolumeMean = 10
deltaTVolumeSigma = 0.2
undercoolingVelocity = -0.1



ani = RunAndShowgg(nx,ny,nGrainsSurface,deltaTSurfaceMean,deltaTSurfaceSigma\
                   ,nGrainsVolume,deltaTVolumeMean,deltaTVolumeSigma, undercoolingVelocity, niter=maxiter)
HTML(ani.to_jshtml())

### Case two: seeds appear at the same time on the surface and in the bulk

How do you think this will differ from the previous case?

In [ ]:
[nx,ny] = [100,100]
maxiter = 120
nGrainsSurface = 100
nGrainsVolume = 100
deltaTSurfaceMean = 4
deltaTSurfaceSigma = 0.1
deltaTVolumeMean = 4
deltaTVolumeSigma = 0.1
undercoolingVelocity = -0.1



ani = RunAndShowgg(nx,ny,nGrainsSurface,deltaTSurfaceMean,deltaTSurfaceSigma\
                   ,nGrainsVolume,deltaTVolumeMean,deltaTVolumeSigma, undercoolingVelocity, niter=maxiter)
HTML(ani.to_jshtml())

### Case three: seeds appear first in the bulk

What do you think the simulation will look like?

In [ ]:
[nx,ny] = [100,100]
maxiter = 120
nGrainsSurface = 100
nGrainsVolume = 80
deltaTSurfaceMean = 8
deltaTSurfaceSigma = 0.1
deltaTVolumeMean = 0.5
deltaTVolumeSigma = 0.1
undercoolingVelocity = -0.1



ani = RunAndShowgg(nx,ny,nGrainsSurface,deltaTSurfaceMean,deltaTSurfaceSigma\
                   ,nGrainsVolume,deltaTVolumeMean,deltaTVolumeSigma, undercoolingVelocity, niter=maxiter)
HTML(ani.to_jshtml())